In [ ]:
%%configure
{"vCores": 8}

In [ ]:
!pip install -q tpchgen-cli
!pip install -q daft

In [ ]:
sf = 1000
lh = 'data'
ws = 'disccache'
BASE_OUTPUT_DIR=f"/lakehouse/default/Files/CH{sf}"

**<mark>Generate Data</mark>**

In [ ]:
%%time
shell_script = f"""
SCALE_FACTOR={sf}
TABLES=("customer" "lineitem" "nation" "orders" "part" "partsupp" "region" "supplier")
FORMAT="parquet"
BASE_OUTPUT_DIR="{BASE_OUTPUT_DIR}"
echo "Using SCALE_FACTOR=${{SCALE_FACTOR}}"
echo "Output base directory: ${{BASE_OUTPUT_DIR}}"
echo "Format: ${{FORMAT}}"
echo "Generating tables: ${{TABLES[@]}}"
for table in "${{TABLES[@]}}"; do
    OUTPUT_DIR="${{BASE_OUTPUT_DIR}}/${{table}}"
    echo "Generating data for table: ${{table}} into ${{OUTPUT_DIR}}"
    tpchgen-cli --tables "${{table}}" --scale-factor "${{SCALE_FACTOR}}" --output-dir "${{OUTPUT_DIR}}" --format "${{FORMAT}}"
done
"""
if not os.path.isdir(BASE_OUTPUT_DIR):
    get_ipython().system(shell_script)
else:
    print(" folder exists")

**<mark>split the big parquet file</mark>**

In [ ]:
%%time
import daft
for tbl in ['region','nation','lineitem','orders','partsupp','part','customer','supplier']:
 print(tbl)
 path =f'/lakehouse/default'
 daft.read_parquet(path +f'/Files/CH{sf}/{tbl}/*.parquet').write_parquet(path +f'/Tables/CH{sf}/{tbl}',write_mode="overwrite")

**<mark>convert to Delta</mark>**

In [ ]:
%%time
from   deltalake   import DeltaTable ,convert_to_deltalake
for tbl in ['region','nation','lineitem','orders','partsupp','part','customer','supplier']: 
 print(tbl)
 convert_to_deltalake(
     f'abfss://{ws}@onelake.dfs.fabric.microsoft.com/{lh}.Lakehouse/Tables/CH{sf}/{tbl}',
     storage_options={"bearer_token": notebookutils.credentials.getToken('storage'), "use_fabric_endpoint": "true"}
 )